<a href="https://colab.research.google.com/github/unmeshvrije/Pytensorflow/blob/master/Deep_learning_for_knowledge_graphs_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning for Knowledge Graphs from scratch

slides: https://bit.ly/2PIcGez

This notebook shows some of the basics of deep learning in pytorch for machine learning with knowledge graphs. 

In [0]:
# we will be building everything from scratch on top of the pytorch framework
import torch

In [0]:
# these libraries aren't installed by default in colab. The exclamation point
# executes a command line statement
!pip install rdflib wget

In [0]:
# Utility functions for later. Execute this cell, but ignore what's in it.

from collections import Counter

def generate_likes(num_users, num_items):
  """
  Generate fake user/item data
  """

  users = torch.randn(num_users, 2)
  items = torch.randn(num_items, 2)

  return (torch.mm(users, items.t()) > 0.7).nonzero()

def load_aifb():
    """
    Loads a knowledge graph dataset.
    
    :return: A tuple containing the graph data, and the classification test and train sets.
    """
    online = 'https://github.com/pbloem/gated-rgcn/blob/master/data/aifb/aifb_stripped.nt.gz?raw=true'
    file = 'aifb.nt.gz'

    wget.download(online, out=file) # download the file from github

    
    # -- Parse the data with RDFLib
    graph = rdf.Graph()

    if file.endswith('nt.gz'):
        with gzip.open(file, 'rb') as f:
            graph.parse(file=f, format='nt')
    else:
        graph.parse(file, format=rdf.util.guess_format(file))

    # -- Collect all node and relation labels
    nodes = set()
    relations = Counter()

    for s, p, o in graph:
        nodes.add(str(s))
        nodes.add(str(o))
        relations[str(p)] += 1

    i2n = list(nodes) # maps indices to labels
    n2i = {n:i for i, n in enumerate(i2n)} # maps labels to indices


    i2r =list(relations.keys())
    r2i = {r: i for i, r in enumerate(i2r)}

    edges = {}

    # -- Collect all edges into a list of triples
    #    (only storing integer indices)
    triples = [(n2i[str(s)], r2i[str(p)], n2i[str(o)]) for s, p, o in graph]


    return triples, (n2i, i2n), (r2i, i2r)


# Pytorch crash course

We'll start with some basic aspects of pytorch. Pytorch is built around vectors, matrices and their higher-dimensional equivalents _tensors_. Here's how to create and manipulate them:

In [0]:
# make a matrix filled with (normally distributed) random numbers
torch.randn(3, 4)

# For higher-dimensional matrices (aka tensors), just add more dimensions:
# torch.randn(3, 4, 2)

In [0]:
# make some more
a, b = torch.randn(3, 4), torch.rand(3, 4)
# note the different dimensions
c = torch.randn(4, 3)

# addition, multiplication, etc are all element-wise
summed = a + b
mult = a * b
power = a ** 2
sine = torch.sin(a)

# _matrix_ multiplication is done through torch.mm
mmult = torch.mm(a, c)

# Note that the following lines would both give an error. Why?
# mult = a * c
# torch.mm(a, b)

In [0]:
# Indexing and slicing
print(a)

print()
print('element at the first row and third column of a:', a[0, 2]) # note: zero-indexing
print('   first row of a:', a[0, :])
print('first column of a:', a[:, 0])

# You can also use basic slicing syntax; i:j refers to the range from i to j
# (more precisely, i is the first element included, j is the first element 
#  excluded)
print('middle two elements of each row of a:\n', a[:, 1:3])

## Backpropagation

All learning in deep learning is done through some form of _gradient descent_. To compute the gradient for some operation automatically, pytorch provides the autograd package. Autograd allows you to keep track of all the operations you perform to get to some final value (a scalar) so that you can then compute the gradient automatically, through the backpropagation algorithm. Here's how it works:




In [0]:
from torch.autograd import Variable

# First we wrap the a and b matrices in a Variable object. This ensures that 
# pytorch remembers how the variable was created


av = Variable(a, requires_grad=True) # tell pytorch to remember the gradient for this one
bv = Variable(b)

# perform some operations that result in a single scalar 'out'
multv = av * bv
out = (multv ** 2).sum()

# execute the backpropagation algorithm: compute all required gradient of 'out'
# with respect to the variables in the operation
out.backward()

# show the gradient of out with respect to a
print(av.grad)

We can now build a classic multilayer perceptron very easily. All we need to do is execute the forward pass (from input to output), compute the loss (how close the output is to the target) and backpropagate the loss.

We'll build a neural net for a 3D input vector $(x_1, x_2, x_3)$ to a single target value $t = {x_1} + {x_2} + x_1$. We'll give the network a hidden layer with 6 units and a ReLU activation. We'll call the weight matrices of the first and second layers w and v. 

In [0]:
# The data (just one example)
x = torch.randn(3)
t = x[0] + x[1] + x[2]

# wrap in variables (no gradient required over these)
x = Variable(x)
t = Variable(t)

# initialize weights
w = Variable(torch.randn(6, 3), requires_grad=True)
v = Variable(torch.randn(1, 6), requires_grad=True)

# execute the network
h = torch.mv(w, x)   # matrix-vector multiplication
h = torch.relu(h)    # activation
y = torch.mv(v, h)   # second layer

# compute the loss (squared error loss)
loss = (t - y) ** 2

# backpropagate
loss.backward()

# gradient on the w parameters:
print(w.grad)

# one learning step
w.data = w.data - 0.001 * w.grad

To make learning easier, pytorch provides some tools that perform these steps for us. The first is the _optimizer_. This is an object that performs the gradient step for us. The basic gradient step shown above is performed by the SGD optimizer, but many slightly more clever optimizers are available. Let's try out the Adam optimizer, which is a very popular choice.



In [0]:
from torch.optim import Adam, SGD

# reset the parameters, bigger hidden layer
w = Variable(torch.randn(6, 3), requires_grad=True)
v = Variable(torch.randn(1, 6), requires_grad=True)

# we initialize the optimizer by telling it the learning rate and which parameters 
# to update
optimizer = Adam(lr=0.0001, params=[w, v])

# next, we write a training loop, showing the network new data each iteration
# (randomly generated every time)

for i in range(5000):
  x = torch.randn(3)
  t = x[0] + x[1] + x[2]

  x, t = Variable(x), Variable(t)
  
  # clear all existing gradients
  optimizer.zero_grad()
  
  # compute the network
  # execute the network
  h = torch.mv(w, x)   # matrix-vector multiplication
  h = torch.relu(h)    # activation
  y = torch.mv(v, h)   # second layer

  # compute the loss (squared error loss)
  loss = (t - y) ** 2

  # backpropagate
  loss.backward()
  
  if i % 250 == 0:
    print('iteration {}, loss {:.4}'.format(i, loss.item()))
    # print(w)
    
  # take a gradient descent step
  optimizer.step()
  


## Neural network modules

The second pytorch package that we'll look at is ```torch.nn```. This provides full featured _modules_ implementing the basic neural network layers. The fully connected layer we've used above is called linear. It takes care of initializing and remembering the parameters (including a bias).

All modules in the nn package expect the data in _batches_ of multiple instance. The model is computed in parallel for all instances in the batch. This makes things much quicker on parallel hardware and helps to stabilize learning.

In [0]:
import torch.nn as nn

batch_size = 250

# Create the two layers
one = nn.Linear(3, 6)    
two = nn.Linear(6, 1)

# you can just apply the layers by calling, for instance, one(x), but it's 
# helpful to combine them into a module that does all this. We can create our 
# own module object of we have a complex model, but if our 
# model is just a single chain of layers, with the output of each becoming the 
# input of the next, we can use the Sequential model

model = nn.Sequential(
    one, 
    nn.ReLU(),
    two
)

# we initialize the optimizer by telling it the learning rate and which parameters 
# to update
optimizer = Adam(lr=0.5, params=model.parameters())

# next, we write a training loop, showing the network new data each iteration
# (randomly generated every time)

for i in range(0, 50000, batch_size):
  x = torch.randn(batch_size, 3)
  t = x[:, 0] + x[:, 1] + x[:, 2]

  x, t = Variable(x), Variable(t)
  
  # clear all existing gradients
  optimizer.zero_grad()
  
  # compute the network
  y = model(x)
  
  # compute the loss (squared error loss)
  loss = ((t - y[:, 0]) ** 2).sum() # we're using batches now, so we should sum the 
                                    # loss over all elements in the batch

  # backpropagate
  loss.backward()
  
  if i % 2500 == 0:
    print('iteration {}, loss {:.4}'.format(i, loss.item()))
    # print(w)
    
  # take a gradient descent step
  optimizer.step()
  

This may seem like a lot of hassle, just to train a simple feedforward net. Something that in other frameworks (like keras or sklearn) can be done with a few lines of code. 

That's true, but the benefit is that we have a very flexible setup. We can compute any sequence of linear algebra operations on our data, and so long as it results in a single loss value, we can call ```loss.backward()``` to compute the gradient and optimize the parameters.

# A simple embedding model: matrix factorization

In [0]:
import random

num_users, num_items = 20, 40

likes = generate_likes(num_users, num_items)
num_likes = likes.size(0)

# model parameters: embeddings for the users and for the items
users = Variable(torch.randn(num_users, 2), requires_grad=True)
items = Variable(torch.randn(num_items, 2), requires_grad=True)

opt = Adam(lr = 0.0005, params=[users, items])

for it in range(5000):
  opt.zero_grad()
  
  # pick a random positive pair
  u, i = likes[random.randint(0, num_likes-1), :]
  
  # get the embeddings 
  user, item = users[u, :], items[i, :]
  
  # compute the dot product
  pdot = torch.dot(user, item)
  
  # pick a random negative pair
  u, i = random.randint(0, num_users-1), random.randint(0, num_items-1)
  
  # get the embeddings 
  user, item = users[u, :], items[i, :] #
  
  # compute the dot product
  ndot = torch.dot(user, item)
  
  loss = ndot - pdot 
  
  loss.backward()
  
  opt.step()
  
  if it % 250 == 0:
    print(it, loss.item())

  
  

It's a little difficult to see whether the loss actually goes down. We'll fix that later, when we move to knowledge graphs, by processing the data in batches (so the loss is averaged over multiple examples)

# Knowledge graphs



In [0]:
!pip install rdflib wget

In [0]:
import rdflib as rdf
import wget, gzip

In [0]:
# Load a small example knowledge graph (the AIFB dataset)
# -- this function loads the graph and maps all urls (nodes and relations) to 
#    unique integers, that we can work with in a machine learning context.
# -- triples is a list of integer triples. n2i maps nodes to integer indices and 
#    similar for the others
triples, (n2i, i2n), (r2i, i2r) = load_aifb()

num_nodes, num_rels, num_links = len(i2n), len(i2r), len(triples)

print(num_nodes, num_rels, num_links)
triples[:10]



# RESCAL

_NB: The proper RESCAL algorithm has some more features than this. We're just illustrating the RESCAL score function._

In [0]:
from torch import nn

# set up the model parameters

k = 16 # embedding dimension

# -- node embeddings
nodes = nn.Parameter(torch.randn(num_nodes, k))   # single k-vector per node
# -- relation embeddings
rels  = nn.Parameter(torch.randn(num_rels, k, k)) # k-by-k matrix per relation

# (nn.Parameter is basically the same as Variable, but gradients are turned on
#  by default, together with some other helpful features.)

opt = Adam(lr = 0.001, params=[nodes, rels])

for it in range(5000):
  opt.zero_grad()
  
  # pick a random positive pair
  s, p, o = random.choice(triples)
  
  # get the embeddings, compute the score
  sub, pred, obj = nodes[s, :], rels[p, :, :], nodes[o, :]
  pscore = sub.dot(torch.mv(pred, obj))
  
  # pick a random negative pair 
  # -- we do this by choosing an existing triple and corrupting either the left 
  #    or right node
  s, p, o = random.choice(triples)
  if random.choice([True, False]):
    s = random.randint(0, num_nodes - 1)
  else:
    o = random.randint(0, num_nodes - 1)
  
  # get the embeddings 
  sub, pred, obj = nodes[s, :], rels[p, :, :], nodes[o, :]
  # compute the bilinear product, compute the score
  nscore = sub.dot(torch.mv(pred, obj))
  
  loss = nscore - pscore
  
  loss.backward()
  
  opt.step()
  
  if it % 250 == 0:
    print(it, loss.item())


# DistMult

The RESCAL score function works pretty well, but the relation embeddings tend to be overparametrized, compared to those of the nodes. The _DistMult_ score functionfixes this by forcing the relation embeddings to be diagonal (i.e. zero everywhere but the diagonal), so that the relations are embedded in a vector as well.

This means that the score function for a triple $(s, p, o)$ with embeddings $e_s$, $e_p$, $e_o$ becomes:

$$\text{score} = {e_s}^T \text{diag}(e_r)e_o = \text{sum}(e_s \otimes e_r \otimes e_o) $$

Where $\otimes$ represents element-wise multiplication.

Below, we'll implement DistMult. We'll also add batching to the code, so that we process multiple randomly selected triples per iteration.

In [0]:
from torch import sigmoid, log

# set up the model parameters

batch_size = 64
k = 16 # embedding dimension

# -- node embeddings
nodes = nn.Parameter(torch.randn(num_nodes, k))   # single k-vector per node
# -- relation embeddings
rels  = nn.Parameter(torch.randn(num_rels, k)) # k-by-k matrix per relation

# (nn.Parameter is basically the same as Variable, but gradients are turned on
#  by default, together with some other helpful features.)

opt = Adam(lr = 0.00001, params=[nodes, rels])

for it in range(200_000): # decent looking results at abt 100k to 500k
  opt.zero_grad()
  
  # select a batch of positive pairs
  positives = random.sample(triples, batch_size)
  # -- split them into indices for s, p and o
  s, p, o = [s for s, _, _ in positives], [p for _, p, _ in positives], [o for _, _, o in positives]
  
  # get the embeddings, compute the score
  # -- pytorch allows us to use a list of integers to select multiple rows
  sub, pred, obj = nodes[s, :], rels[p, :], nodes[o, :]
  
  pscore = (sub * pred * obj).sum(dim=1) # we sum over dimension 1, so we get a 
                                         # score for each instancein the batch 
  
  # pick a random negative pair 
  # -- we do this by choosing an existing triple and corrupting either the left 
  #    or right node
  
  # -- sample some triples
  negatives = random.sample(triples, batch_size)
  # -- split them into indices for s, p and o
  s, p, o = [s for s, _, _ in positives], [p for _, p, _ in positives], [o for _, _, o in positives]  
  
  s, p, o = torch.tensor(s), torch.tensor(p), torch.tensor(o)
  
  indices = torch.rand(batch_size) > 0.5 # vector of random bits
  
  # vector of random node indices
  nwnodes = (torch.rand(batch_size) * num_nodes).floor().long()

  # corrupt sub at the chosen indices, corrupt obj at the other indices
  s[ indices] = nwnodes[ indices]
  o[~indices] = nwnodes[~indices]
  
  # get the embeddings 
  sub, pred, obj = nodes[s, :], rels[p, :], nodes[o, :]

  # compute the  the score
  nscore = (sub * pred * obj).sum(dim=1)
  
  # plain loss
  # loss = nscore.sum() - pscore.sum()
  
  # log loss
  loss = log(sigmoid(nscore)).sum() - log(sigmoid(pscore)).sum()
  
  loss.backward()
  
  opt.step()
  
  if it % 20_000 == 0:
    print(it, loss.item())
    
# NB: This one may take a while ...

In [0]:
# pick a random triple, and print its labels

i = random.randint(0, len(triples)-1)
s, p, o = triples[i]

print('using triple {}:'.format(i), i2n[s], i2r[p], i2n[o])

scores = []


sub = nodes[s, :]
pred = rels[p, :]

for o in range(num_nodes):

  obj = nodes[o, :]
  
  score = (sub * pred * obj).sum().item()
  
  scores.append((o, score))

# sort by score
scores.sort(key = lambda x : - x[1])

# print top 10
for o, score in scores[:10]:
  print('{},\t {:.4}'.format(i2n[o], score))